# Load libraries

In [13]:
import os
import numpy as np
import pandas as pd

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from ipywidgets import *
from IPython.display import clear_output, display
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import qgrid

import helpers.research_helpers as app

CSV_SEPARATOR = ','     # '\t'

print('Notebook initialized.')

Notebook initialized.


# Load data

In [2]:
### Load membership list

df_all_active_roster = app.load_csv('../data/All Active Roster.csv')

# Filter by... membership_items

In [3]:
### Please select those membership_items you would like to keep.
### The default list (keep_records.KEEP_LIST_MEMBERSHIP_ITEMS) is already selected.

# Allow user's to select which membership_item groups, if any, they want to exclude
qgrid_sheet_to_keep = app.keep_grid_show_filter(df_all_active_roster, app.COL_MEMBERSHIP_ITEM, app.COL_MEMBER_ID, app.KEEP_LIST_MEMBERSHIP_ITEMS)
qgrid_sheet_to_keep

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Apply changes

In [4]:
### Apply filtering

# To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_list = app.keep_grid_apply_filter(qgrid_sheet_to_keep, app.COL_MEMBERSHIP_ITEM, df_all_active_roster)

The filter was successfully applied to the data frame.
There are 275490 out of 302352 email addresses available for use.


# Filter by... membership expiration date range

In [5]:
### Please select those expiration groups you would like to keep.
### The default list (keep_records.KEEP_LIST_EXPIRES_IN) is already selected.

### Calculate Expiration Range based off of EXPIRY_DATE and store in EXPIRY_RANGE_CALCULATED
df_filtered_list = app.assign_expiration_range(df_filtered_list)

# Allow user's to select which expiration groups, if any, they want to exclude
qgrid_sheet_to_keep = app.keep_grid_show_filter(df_filtered_list, app.COL_EXPIRY_RANGE_CALCULATED, app.COL_MEMBER_ID, app.KEEP_LIST_EXPIRES_IN)
qgrid_sheet_to_keep

Assigned expiration range.


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Apply changes

In [6]:
### Apply filtering

# To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_list = app.keep_grid_apply_filter(qgrid_sheet_to_keep, app.COL_EXPIRY_RANGE_CALCULATED, df_filtered_list)

The filter was successfully applied to the data frame.
There are 178009 out of 275490 email addresses available for use.


# Calculate... Manager/Worker

In [7]:
### Calculate Title based off of JOB_TITLE, BOSS_TITLE, and NUM_OVERSEEN and store in TITLE_CALCULATED
df_filtered_list = app.assign_title(df_filtered_list)

df_test = df_filtered_list.filter([app.COL_JOB_TITLE, app.COL_BOSS_TITLE, app.COL_NUM_OVERSEEN, app.COL_TITLE_CALCULATED])
qgrid_widget = qgrid.show_grid(df_test, show_toolbar=False)
qgrid_widget

Assigned Titles.


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Calculate... Pub/Private Sector

In [8]:
### Calculate Pub/Private Sector based off of ORG_TYPE, INDUSTRY, and EMAIL_DOMAIN and store in PUB_PRIVATE_CALCULATED
df_filtered_list = app.assign_pub_private_sector(df_filtered_list)

df_test = df_filtered_list.filter([app.COL_ORG_TYPE, app.COL_INDUSTRY, app.COL_EMAIL_DOMAIN, app.COL_PUB_PRIVATE_CALCULATED])
qgrid_widget = qgrid.show_grid(df_test, show_toolbar=False)
qgrid_widget

Assigned Pub/Private Sector.


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [10]:
#df_filtered_list[app.COL_JOB_TITLE] = df_filtered_list[app.COL_JOB_TITLE].fillna('')

qgrid_sheet_to_keep = app.keep_grid_show_filter(df_filtered_list, app.COL_JOB_TITLE, app.COL_MEMBER_ID, app.KEEP_LIST_JOB_TITLES)
qgrid_sheet_to_keep

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Filter by... job_title

## Apply changes

In [11]:
### Apply filtering

# To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_list = app.keep_grid_apply_filter(qgrid_sheet_to_keep, app.COL_JOB_TITLE, df_filtered_list)

The filter was successfully applied to the data frame.
There are 170656 out of 178009 email addresses available for use.


# Filter by... Survey Recency

## Apply changes

# Save Output

In [18]:
### Add code to save project and data files to a directory of the user's choosing
project_name_lbl = widgets.Label('Project Name: ')
project_name_txt = widgets.Text()
display(project_name_lbl)
display(project_name_txt)

Label(value='Project Name: ')

Text(value='')

In [22]:
project_name_results_lbl = widgets.Label()
project_name_results_lbl.value = 'Your data will be saved to ../s_drive/' + project_name_txt.value + '/'
display(project_name_results_lbl)

Label(value="Your data will be saved to ../s_drive/Brad's Test Project/")